In [1]:
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '16g'), ('spark.driver.memory','15g'),("spark.driver.maxResultSize","0"),("spark.executor.cores", '4'),("spark.dynamicAllocation.enabled","False")])

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,application_1649178090023_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.storagelevel import StorageLevel

from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Transformer, classification
from pyspark.ml import tuning
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
cannot import name 'TrainValidationSplit' from 'pyspark.ml.evaluation' (/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/evaluation.py)
Traceback (most recent call last):
ImportError: cannot import name 'TrainValidationSplit' from 'pyspark.ml.evaluation' (/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/evaluation.py)



In [4]:
df = sqlContext.read.parquet("hdfs:///NLP-parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = df.withColumnRenamed("stars","label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- rawFeatures: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

In [7]:
#gbt = Classifier(featuresCol='features',labelCol="stars")
#model = DecisionTree.trainClassifier(sc.parallelize(df), 5, {})
#NB = NaiveBayes(featuresCol='features', labelCol='label', smoothing=1.0, modelType='multinomial', thresholds=None, weightCol=None)
#rf = classification.DecisionTreeClassifier(featuresCol='features',maxDepth=3, labelCol="stars",predictionCol='prediction', seed=42)

#lr = classification.LogisticRegression(maxIter=100, regParam=0.03, elasticNetParam=0.1, tol=1E-6, standardization=True, family="binomial")
lr = classification.LogisticRegression()
ovr = classification.OneVsRest(classifier=lr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

paramGrid = tuning.ParamGridBuilder().addGrid(lr.maxIter, [50,75,100]) \
                              .addGrid(lr.regParam, [0.03, 0.04, 0.05, 0.06]) \
                              .addGrid(lr.elasticNetParam, [0.1,0.2,0.3,0.4]) \
                              .build()
evaluator = MulticlassClassificationEvaluator()
valid = tuning.TrainValidationSplit(estimator=ovr,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             trainRatio=0.50)
model = valid.fit(df)
result = model.bestModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-15:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1190



In [11]:
#rf_model = rf.fit(df)

#lrModel = ovr.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'requirement failed: LinearSVC only supports binary classification. 1 classes detected in LinearSVC_854a91718e66__labelCol'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/classification.py", line 1838, in _fit
    models = pool.map(trainSingleClass, range(numClasses))
  File "/usr/lib64/python3.7/multiprocessing/pool.py", line 268, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/usr/lib64/python3.7/multiprocessing/pool.py", line 657, in get
    raise self._value
  File "/usr/lib64/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib64/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/classification.py", line 1834, in t

In [9]:
result = lrModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'lrModel' is not defined
Traceback (most recent call last):
NameError: name 'lrModel' is not defined



In [11]:
result.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- rawFeatures: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = true)

In [12]:
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+----------+
|         rawFeatures|label|            features|prediction|
+--------------------+-----+--------------------+----------+
|(65536,[2579,2742...|  3.0|(65536,[2579,2742...|       3.0|
|(65536,[3067,4072...|  3.0|(65536,[3067,4072...|       3.0|
|(65536,[3923,4943...|  3.0|(65536,[3923,4943...|       3.0|
|(65536,[2346,3923...|  1.0|(65536,[2346,3923...|       1.0|
|(65536,[28848,307...|  5.0|(65536,[28848,307...|       2.0|
|(65536,[1431,3067...|  2.0|(65536,[1431,3067...|       3.0|
|(65536,[1431,3923...|  2.0|(65536,[1431,3923...|       4.0|
|(65536,[6908,7792...|  3.0|(65536,[6908,7792...|       5.0|
|(65536,[5325,7051...|  4.0|(65536,[5325,7051...|       5.0|
|(65536,[7284,1481...|  5.0|(65536,[7284,1481...|       5.0|
|(65536,[2182,6908...|  1.0|(65536,[2182,6908...|       1.0|
|(65536,[1038,1969...|  5.0|(65536,[1038,1969...|       5.0|
|(65536,[714,895,1...|  2.0|(65536,[714,895,1...|       1.0|
|(65536,[7284,2014...|  

In [13]:

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5292433626942162

In [7]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…